# Data Cleaning

#### The following notebook describes the process in reaching our final dataset

In [39]:
import numpy as np
from scipy import sparse
from scipy.stats import mstats
import re
import matplotlib
import pandas as pd
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.cm as cmx
import matplotlib.colors as colors
import datetime as dt
import math
from sklearn.linear_model import Lasso as Lasso_Reg
import numpy as np 
import sklearn.preprocessing as Preprocessing
from sklearn.preprocessing import StandardScaler as Standardize
%matplotlib inline
from sklearn.linear_model import LinearRegression as Lin_Reg
from sklearn.linear_model import Ridge as Ridge_Reg

### Clean Calendar Dataset

The calendar dataset gives the price of listings over the course of a year. In order to clean this dataset, we first renamed the columns with more appropriate labels. We then stripped the dates and prices of their extraneous characters so they were easier to work with. Finally, we converted the column types to apropriate dtypes and filtered out columns with no information. Our calendar dataset, which was too large to upload on Github, is pulled from the following website: file:///Users/Mirai/Downloads/Airbnb_Pricing_Prediction%20(20).pdf

In [2]:
# open calendar csv file
calendar = pd.read_csv('calendar.csv', delimiter = '\t') 

# rename columns 
calendar["listing"], calendar['date'], calendar['available'], calendar['price'] = zip(*calendar['listing_id,"date","available","price"'].str.split(',').tolist())
calendar = calendar.drop(['listing_id,"date","available","price"'], 1)

# change column contents to be more workable format
calendar['date'] = calendar['date'].map(lambda x: x.lstrip('"').rstrip('"'))
calendar['price'] = calendar['price'].map(lambda x: x.lstrip('$').rstrip('.'))

# change column dtypes 
calendar['price'] = pd.to_numeric(calendar['price'])
calendar['date'] = pd.to_datetime(calendar['date'])
calendar['listing'] = pd.to_numeric(calendar['listing'])

# filter out the columns with no prices 
calendar=calendar[calendar['available'] == 't']

### Clean Listings Dataset

The listings dataset is our main dataset for this project. It contains thousands of rows of listing information for Airbnbs in New York City. We attempted to clean the data many different ways before settling on our final dataset. One notable technique we tried was KNN. Ultimately, this was unsuccessful because most rows had some NaN values and it was therefore difficult to find similar listings. KNN ended up being too costly and ineffective.

Below, we outline the process that ended up working best for us: 
First, we dropped the columns with null values that added no values to the dataset. We then dropped null values that could not be salvaged (things that could not be filled in based off of any known technique or our intuition such as ID or property type).
Next, we changed the format of two variables (price and extra people) to integers rather than objects so we could easily perform statistical procedures with the information provided. The zipcodes were given in their extended form, so we decided to only use the first five numbers.
For missing weekly and monthly prices and regular prices, we used scaled nightly price to fill in the missing data. Also, for all the pricing data, we converted the format to float values. 

In [3]:
# open dataset with information and pricing for each listing 
df = pd.read_csv('listings.csv',encoding='latin1')

In [4]:
# drop columns with null values that don't add important information to dataset
df = df.drop(['country', 'neighbourhood', 'square_feet', 'state'], 1)

# drop null values that can't be salvaged 
df = df[(pd.notnull(df['id']))&(pd.notnull(df['host_id']))&(pd.notnull(df['zipcode']))&(pd.notnull(df['latitude']))]
df = df[(pd.notnull(df['longitude']))&(pd.notnull(df['bathrooms']))&(pd.notnull(df['bedrooms']))&(pd.notnull(df['beds']))]
df = df[(pd.notnull(df['property_type']))&(pd.notnull(df['price']))&((df.number_of_reviews!=0)&(pd.notnull(df.review_scores_rating)))]

# reset index after dropping certain rows
df = df.reset_index(drop=True)

# change format of prices and extra_people to integer rather than object
prices = []
extra_people = []
for i in range(len(df)):
    price = int(float(str(df['price'][i]).replace('$', '').replace(',', '')))
    extra_person = int(float(str(df['extra_people'][i]).replace('$', '').replace(',', '')))
    prices.append(price)
    extra_people.append(extra_person)

df['price'] = prices
df['extra_people'] = extra_people

# only keep first five numbers of zipcode 
zipcodes = []
for i in range(len(df)):
    zipcode = df['zipcode'][i][:5]
    zipcodes.append(zipcode)
df['zipcode'] = zipcodes

# fill the null values in weekly_prices and monthly_prices column
# change the dtype from object to integer in these two columns
wprices = []
mprices = []

for i in range(len(df)):
    if pd.isnull(df['weekly_price'][i]):
        wprice = df['price'][i] * 7
    elif pd.notnull(df['weekly_price'][i]):
        wprice = int(float(str(df['weekly_price'][i]).replace('$', '').replace(',', '')))
    wprices.append(wprice)
    if pd.isnull(df['monthly_price'][i]):
        mprice = df['price'][i] * 30
    elif pd.notnull(df['monthly_price'][i]):
        mprice = int(float(str(df['monthly_price'][i]).replace('$', '').replace(',', '')))
    mprices.append(mprice)
    
df['weekly_price'] = wprices
df['monthly_price'] = mprices

df = df.dropna(axis=0)
df = df.reset_index(drop=True)

### Adding Sentiment Analysis

We performed sentiment analysis using a dataset that provided reviews on different listings. More details about sentiment analysis are shown in another notebook, as there is a good deal of cleaning, exploration, and analysis that occurred on that notebook alone, but here we show how sentiment scores were included into our master dataset. Additionally, we added the 'host_since', 'first_review', and 'last_review' columns from the sentiment dataset, because in that dataset cleaning, we had altered these columns (rather than dates, we made them "time", i.e.: seconds, to so that they were in integer, rather than object, format). More to come in that notebook. 

In [5]:
# drop the original host_since, first_review, last_review
df = df.drop(['host_since', 'first_review', 'last_review'], 1)

# add in new csv with the sentiment columns and converted host_since, first_review, last_review columns
sentiment = pd.read_csv('time_sentiment.csv')

host_since = sentiment['host_since'] 
first_review = sentiment['first_review'] 
last_review = sentiment['last_review'] 
sentiments = sentiment['sentiment']

frames = [host_since, first_review, last_review, sentiments]
final_sentiment = pd.concat(frames, 1)
final_sentiment = final_sentiment.dropna(axis=0, how='any')
frames_new = [df, final_sentiment]

df_final = pd.concat(frames_new, 1)

### Incorporating Indicator for Holidays

We later explain some explorations that were done on our calendar dataset. However, here we show how we added a column of indicators to the master dataset for each holiday; a listing received a "1" on a certain column if its price fluctuated on the day of the holiday. 

In [22]:
# array of most important holidays 
unique_ids = df_final['id'].unique()
unique_dates = ['2015-01-01', '2015-07-04', '2015-12-25', '2015-12-31']

# empty arrays which we will add our indicators to 
new_years = []
july_4th = []
christmas = []
eve = []
holidays = [new_years, july_4th, christmas, eve]

# add an indicator for each row if a listing's price changed on any of these holidays 
for i in range(len(unique_ids)):
    real_price = df_final['price'][i]
    list_prices = calendar[calendar['listing'] == unique_ids[i]]
    # indicator column for each holiday
    for j in range(len(unique_dates)):
        price_holiday = list_prices['price'][list_prices['date'] == unique_dates[j]]
        if len(price_holiday) > 0:
            # check if price changes on that holiday 
            if price_holiday.iloc[0] != real_price:
                holidays[j].append(1)
            else:
                holidays[j].append(0)
        else:
            holidays[j].append(0)

In [23]:
# add column of indicators for each holiday to the dataset
df_final['New Years'] = new_years
df_final['July 4th'] = july_4th
df_final['Christmas'] = christmas
df_final['New Years Eve'] = eve

### Adding "Nearest Attractions" Feature

More details are shown in a separate notebook, but here we add a count of the best attractions in NY that the listing is close to. 

In [24]:
attractions = pd.read_csv('attractions_added.csv')

In [25]:
attractions = attractions['attraction_count']
df_final['attraction_count'] = attractions

### Standardization of Certain Variables

In a separate dataset, after an iteration of tuning models, we noticed a few features that were skewing our results because they were not standardized. Hence, we added this step of 'feature engineering' to our final dataset, in which we standardized/winsorized certain features in our master dataset.

In [40]:
#standardize the new data
for feat in ['host_since','first_review','last_review']:
    df_final[feat] = (df_final[feat] - df_final[feat].mean()) / (df_final[feat].max() - df_final[feat].min())
df_final['maximum_nights'] = mstats.winsorize(df_final['maximum_nights'],limits=(0,0.95))

### Final Dataset!!

In [41]:
df_final.head()

,id,scrape_id,last_scraped,name,picture_url,host_id,host_name,host_picture_url,street,neighbourhood_cleansed,...,host_since,first_review,last_review,sentiment,Christmas,New Years,New Years Eve,Independence Day,July 4th,attraction_count
0,1069266,2.015010e+13,1/2/15,Stay like a real New Yorker!,https://a0.muscache.com/pictures/50276484/larg...,5867023,Michael,https://a2.muscache.com/ic/users/5867023/profi...,"East 53rd Street, New York, NY 10022, United S...",Midtown East,...,0.049161,-0.093997,0.048728,0.283516,0,0,0,0,0,1
1,2061725,2.015010e+13,1/2/15,Option of 2 Beds w Private Bathroom,https://a2.muscache.com/pictures/50650147/larg...,4601412,Miao,https://a0.muscache.com/ic/users/4601412/profi...,"Van Buren Street, Brooklyn, NY 11221, United S...",Bushwick,...,0.010274,0.029741,0.057405,0.437795,0,0,0,0,0,0
2,44974,2.015010e+13,1/3/15,Charming Bright West Village Studio,https://a1.muscache.com/pictures/20489905/larg...,198425,Sara,https://a0.muscache.com/ic/users/198425/profil...,"Greenwich Ave, New York, NY 10011, United States",West Village,...,-0.349120,-0.503387,0.014021,0.308604,0,0,0,0,0,0
3,4701675,2.015010e+13,1/2/15,Charming Apartment in Chelsea,https://a2.muscache.com/pictures/60588955/larg...,22590025,Charles,https://a2.muscache.com/ic/users/22590025/prof...,"West 22nd Street, New York, NY 10011, United S...",Chelsea,...,0.275522,0.164449,0.042220,0.000000,0,0,1,1,0,1
4,68914,2.015010e+13,1/2/15,Calm & Sunny Apt in Carroll Gardens,https://a1.muscache.com/pictures/434004/large.jpg,343302,Price,https://a1.muscache.com/ic/users/343302/profil...,"3rd St, Brooklyn, NY 11231, United States",Carroll Gardens,...,-0.286492,-0.378771,0.044390,0.122600,0,0,0,0,0,0


In [42]:
# all of the columns in our master dataset
df_final.columns

Index([                         u'id',                   u'scrape_id',
                      u'last_scraped',                        u'name',
                       u'picture_url',                     u'host_id',
                         u'host_name',            u'host_picture_url',
                            u'street',      u'neighbourhood_cleansed',
                              u'city',                     u'zipcode',
                            u'market',                    u'latitude',
                         u'longitude',           u'is_location_exact',
                     u'property_type',                   u'room_type',
                      u'accommodates',                   u'bathrooms',
                          u'bedrooms',                        u'beds',
                          u'bed_type',                       u'price',
                      u'weekly_price',               u'monthly_price',
                   u'guests_included',                u'extra_people',
      

In [43]:
df_final.to_csv('airbnb_final.csv')

### Prices

Because we found numerous outliers in the dataset, we believed it would be better to take the log of prices so that prices could be plotted on a more normal distribution. This would also allow us to see relationships between variables and price better. 

In [44]:
log_price = df_final.price.apply(math.log)
df['log_price'] = log_price
df['log_price'].to_csv('log_prices.csv')